# Trabalho de Algoritmos Genéticos

## Problema Do Empacotamento (BPP do inglês *"Bin Packing Problem"*)

Dada uma quantidade inteira e positiva de pacotes/depósitos de capacidade $C$ e um set de $M$ itens  $I = [I_1, \cdots, I_M]$ de tamanhos $S = [S_1,\cdots,S_M]$, o problema consiste em empacotar todos os itens nos pacotes, de modo a não exceder a capacidade $C$, **minimizando** a quantidade $N$ de pacotes utilizados.

## Imports

In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [23]:
import numpy as np
import pandas as pd
import random
from itertools import combinations
import copy

## Modelando o Problema

- Gene: É um pacote.
    - É representado por um set de inteiros referentes à cada cada item;
- Indivíduo: É um conjunto de pacotes, que somam um total de $M$ elementos, cujos respectivos pesos não infringem a capacidade $C$.

### *Fitness*

In [3]:
def fitness(individuo):
    return len(individuo)

### Heurísticas

In [4]:
def FF(individuo, item, C, S):

    if individuo == []:
        individuo.append([item])
        return

    added = False
    for pacote in individuo:
        if sum([x for i, x in enumerate(S) if i in pacote]) + S[item] <= C:
            pacote.append(item)
            pacote.sort()
            added = True
            break

    if not added:
        individuo.append([item])
            

def FFD(individuo, itens, C, S):
    ordenados = [x[0] for x in sorted(
        [(itens[i], S[i]) for i in range(0, len(itens))], 
        key = lambda x: x[1],
        reverse=True)]
    
    for item in ordenados:
        FF(individuo, item, C, S)


def MBS(individuo, item, C, S):

    if individuo == []:
        individuo.append([item])
        return

    added = False
    melhor_diferenca = C
    indice_melhor = 0
    for I, pacote in enumerate(individuo):
        diferenca = C - (sum([x for i, x in enumerate(S) if i in pacote]) + S[item])
        if diferenca < melhor_diferenca and diferenca >= 0:
            melhor_diferenca = diferenca
            indice_melhor = I
            added = True

    if added:
        individuo[indice_melhor] += (item)
        individuo[indice_melhor].sort()
    else:
        individuo.append([item])

def FleszarMBS(individuo, itens, C, S):
    ordenados = [x[0] for x in sorted(
        [(itens[i], S[i]) for i in range(0, len(itens))], 
        key = lambda x: x[1],
        reverse=True)]
    for it in ordenados:
        MBS(individuo, it, C, S)

### Funções Auxiliares

In [62]:
def tamanho(individuo, S):
    tam = 0
    for pacote in individuo:
        tam += sum([x for i, x in enumerate(S) if i in pacote])
    return tam

def P2F(individuo, Tx, C, S, Tb=True):
    if Tb:
        for pacote in Tx:
            tam_pacote_tx = sum([x for i, x in enumerate(S) if i in pacote])
            for pct in individuo:
                tam_pct_indiv = sum([x for i, x in enumerate(S) if i in pct])
                if tam_pct_indiv + tam_pacote_tx <= C:
                    pct += pacote
                    pct.sort()
                    Tx.remove(pacote)
                    break

        if Tx:
            ordenados = [x[0] for x in sorted(
                [(i, sum(
                    [k for j,k in enumerate(S) if  j in i])
                 ) for i in Tx], key = lambda x: x[1], reverse = True)]

            item = ordenados.pop(0)
            individuo += [item]
            Tx.remove(item)
            added = False
            while ordenados:
                melhor_diferenca = C
                indice_melhor = 0
                for I, pacote in enumerate(individuo):
                    diferenca = C - (sum([x for i, x in enumerate(S) if i in pacote]) + sum([y for j, y in enumerate(S) if j in ordenados[0]]))
                    if diferenca < melhor_diferenca and diferenca >= 0:
                        melhor_diferenca = diferenca
                        indice_melhor = I
                        added = True
                if added:
                    individuo[indice_melhor] += ordenados.pop(0)
                    individuo[indice_melhor].sort()
                else:
                    individuo.append(ordenados.pop(0))

    else:
        for item in Tx:
            FF(individuo, item, C, S)
            Tx.remove(item)

    return individuo

def substituicao(filho, Ta, Tb, C, S):
    Tc = [list(x) for x in combinations(Ta, 2)]

    tam_original = tamanho(filho, S)
    lista_candidatos = []

    for indice, pacote in enumerate(filho):
        Sn = []
        Sn += combinations(pacote, 1)
        Sn += combinations(pacote, 2)
        Sn += combinations(pacote, 3)

        for comb in Sn:
            new_filho = copy.deepcopy(filho)

            # remove comb do filho
            new_filho[indice] = [x for x in new_filho[indice] if x not in [comb]]

            # tenta substituir com algum item de Ta
            for pacote in Ta:
                new_Ta = copy.deepcopy(Ta)
                ta_filho = copy.deepcopy(new_filho)
                ta_filho[indice].append(pacote)
                tam_ta = tamanho(ta_filho, S)

                if tam_ta > tam_original and sum([x for i, x in enumerate(S) if i in ta_filho[indice]]) <= C:
                    new_Ta.remove(pacote)
                    candidato = tuple((tam_ta, new_Ta, Tb, ta_filho))
                    if candidato not in lista_candidatos:
                        lista_candidatos.append(candidato)

            # tenta substituir com algum item de Tb
            for pacote in Tb:
                new_Tb = copy.deepcopy(Tb)
                tb_filho = copy.deepcopy(new_filho)
                tb_filho[indice] += pacote
                tam_tb = tamanho(tb_filho, S)

                if tam_tb > tam_original and sum([x for i, x in enumerate(S) if i in tb_filho[indice]]) <= C:
                    new_Tb.remove(pacote)
                    candidato = tuple((tam_tb, Ta, new_Tb, tb_filho))
                    if candidato not in lista_candidatos:
                        lista_candidatos.append(candidato)
                    
            # tenta substituir com algum item de Tc
            for pacote in Tc:
                new_Ta = copy.deepcopy(Ta)
                tc_filho = copy.deepcopy(new_filho)
                tc_filho[indice] += pacote
                tam_tc = tamanho(tc_filho, S)

                if tam_tc > tam_original and sum([x for i, x in enumerate(S) if i in tc_filho[indice]]) <= C:
                    for it in pacote:
                        new_Ta.remove(it)
                    candidato = tuple((tam_tc, new_Ta, Tb, tc_filho))
                    if candidato not in lista_candidatos:
                        lista_candidatos.append(candidato)


    # TODO: verificar condição adicionada
    tam, nTa, nTb, melhor_candidato = max(lista_candidatos, key = lambda x: x[0]) if lista_candidatos else tuple((tam_original, Ta, Tb, filho))

    P2F(melhor_candidato, nTb, C, S)
    
    return P2F(melhor_candidato, nTa, C, S, Tb=False)

### Cruzamento ou *crossover*

In [6]:
def cruzamento(pai1, pai2, C, S):
    return fcruzamento(pai1, pai2, C, S), fcruzamento(pai2, pai1, C, S)

def fcruzamento(pai1, pai2, C, S):
    idx1 = np.random.randint(len(pai1) ,size=2)
    idx2 = np.random.randint(len(pai2) ,size=1)

    filho = []
    for idx in idx1:
        novo_pct = copy.deepcopy(pai1[idx])
        for item in pai1[idx]:
            if any(item in pct_filho for pct_filho in filho):
                novo_pct.remove(item)
        filho += [novo_pct]
    for idx in idx2:
        novo_pct = copy.deepcopy(pai2[idx])
        for item in pai2[idx]:
            if any(item in pct_filho for pct_filho in filho):
                novo_pct.remove(item)
        filho += [novo_pct]
                
            
    
    Ta = []
    Tb = []

    # pacotes remanescentes em pai2
    for pacote in [pct for indice, pct in enumerate(pai2) if indice not in idx2]:
        novo_pacote = []
        for item in pacote:
            if not any(item in o_pct for o_pct in filho):
                novo_pacote.append(item)

        if len(novo_pacote) > 1:
            Tb.append(novo_pacote)
        elif len(novo_pacote) == 1:
            Ta.append(novo_pacote[0])

    return substituicao(filho, Ta, Tb, C, S)

### Mutação

In [70]:
def mutacao(pai, C, S):
    idx = np.random.randint(len(pai), size=np.random.choice([2,3]))

    T = []
    for indice in idx:
        for item in pai[indice]:
            T.append(item)
    T.sort()

    filho = []
    for pacote in [x for i, x in enumerate(pai) if i not in idx]:
        filho += [pacote]

    return substituicao(filho, T, [], C, S)

## Execução

In [7]:
p1 = [[0,1,3,10],[2,9,11],[5,7,13,15],[4,6,14],[8,12]]
p2 = [[3,4,12,15],[6,7,11],[0,9,10],[1,5,8],[2,13,14]]

In [35]:
filho1, filho2 = cruzamento(p1, p2, 40, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
print(filho1)
print(filho2)

[(80, [], [[0, 9, 10], [1, 5], [2, 13]], [[8, 12, 3, 15], [4, 6, 14], [7, 11]]), (81, [], [[3, 15], [1, 5], [2, 13]], [[8, 12, 0, 9, 10], [4, 6, 14], [7, 11]]), (68, [], [[3, 15], [0, 9, 10], [2, 13]], [[8, 12, 1, 5], [4, 6, 14], [7, 11]]), (77, [], [[3, 15], [0, 9, 10], [1, 5]], [[8, 12, 2, 13], [4, 6, 14], [7, 11]]), (68, [], [[3, 15], [0, 9, 10], [2, 13]], [[8, 12], [4, 6, 14, 1, 5], [7, 11]]), (77, [], [[3, 15], [0, 9, 10], [1, 5]], [[8, 12], [4, 6, 14, 2, 13], [7, 11]]), (80, [], [[0, 9, 10], [1, 5], [2, 13]], [[8, 12], [4, 6, 14], [7, 11, 3, 15]]), (81, [], [[3, 15], [1, 5], [2, 13]], [[8, 12], [4, 6, 14], [7, 11, 0, 9, 10]]), (68, [], [[3, 15], [0, 9, 10], [2, 13]], [[8, 12], [4, 6, 14], [7, 11, 1, 5]]), (77, [], [[3, 15], [0, 9, 10], [1, 5]], [[8, 12], [4, 6, 14], [7, 11, 2, 13]])]
[(71, [12], [[2, 11], [4, 6, 14]], [[0, 9, 10, 3], [1, 5, 8], [7, 13, 15]]), (80, [3], [[2, 11], [4, 6, 14]], [[0, 9, 10, 12], [1, 5, 8], [7, 13, 15]]), (81, [3, 12], [[4, 6, 14]], [[0, 9, 10, 2, 11]

In [11]:
mutacao(p1, 40, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

[[0, 0, 1, 2, 3, 3, 9, 10, 11], [5, 7, 13, 15], [4, 6, 10, 14], [8, 12]]

In [ ]:
def main(T=10, G=1000, 
         M=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
         C=40, 
         S=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]):
    
    populacao = []
    
    for _ in range(T):
        individuo = []
        itens = M[:]
        np.random.shuffle(itens)
        for item in itens:
            FF(individuo, item, C, S)
        populacao += [individuo]


    for i in range(0, G):
        idx_random = random.sample(range(len(populacao)), 2)
        xs = []

        xs.extend([populacao[idx_random[0]], populacao[idx_random[1]]])
        x3, x4 = cruzamento(xs[0], xs[1], C, S)
        xs.extend([x3, x4])
        xs.extend([mutacao(xs[0], C, S), mutacao(xs[1], C, S)])
    
        idx_melhor_solucao = 0
        melhores_solucoes = []
    
        while len(melhores_solucoes) < 2:
            for i, elem in enumerate(xs):
                if fitness(elem) < fitness(xs[idx_melhor_solucao]):
                    melhor_solucao = i
    
            melhores_solucoes.append(xs[idx_melhor_solucao])
            xs.pop(i)

        populacao.remove(xs[0])
        populacao.remove(xs[1])
        populacao.extend(melhores_solucoes)


    print(populacao)
    
main()    